In [1]:
import os
config = '.env.dev'
from dotenv import load_dotenv
load_dotenv(config)
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    temperature=0.9,
    model_name=os.environ["OPENAI_MODEL_NAME"], #'text-davinci-003'
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    deployment_name=os.environ["OPENAI_DEPLOYMENT_NAME"]
)


In [2]:
system_prompt_template = """You are chatting with an AI assistant. The assistant is here to help you with any questions you have.

- Assume every request is about the location: {location}

Context:
{context}

History: 
{chat_history}

User Request: {request}

"""

In [3]:
# Now let's use some tools to add some functionality to the agent

from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from tools.geocode import geocodeLocation
from langchain.tools import StructuredTool
system_prompt_template = """You are chatting with an AI assistant. The assistant is here to help you with any questions you have.

- Assume every request is about the location: {location}

Context:
{context}

History: 
{chat_history}

User Request: {request}

"""
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_template),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{request}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

tools = [
    StructuredTool.from_function(geocodeLocation)
]

# https://smith.langchain.com/hub/hwchase17/openai-functions-agent
agent = create_openai_tools_agent(llm, tools, prompt)

data_agent = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    callbacks=[]
)


/Users/andr7059/.pyenv/versions/3.11.10/lib/python3.11/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
data_agent.invoke({
    "request": "where is city hall?", 
    "context": "",
    "location": "Seattle, WA",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `geocodeLocation` with `{'address': 'City Hall, Seattle, WA'}`


(-122.279024, 47.754184)The City Hall in Seattle, WA is located at the following coordinates: 
- Longitude: -122.279024
- Latitude: 47.754184

> Finished chain.


{'request': 'where is city hall?',
 'context': '',
 'location': 'Seattle, WA',
 'chat_history': [],
 'output': 'The City Hall in Seattle, WA is located at the following coordinates: \n- Longitude: -122.279024\n- Latitude: 47.754184'}

In [5]:
data_agent.invoke({
    "request": "where is city hall?", 
    "context": "Provide your response in JSON",
    "location": "Seattle, WA",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `geocodeLocation` with `{'address': 'City Hall, Seattle, WA'}`


(-122.279024, 47.754184)
Invoking: `geocodeLocation` with `{'address': 'City Hall, Seattle, WA'}`


(-122.279024, 47.754184)```json
{
  "city_hall_coordinates": {
    "longitude": -122.279024,
    "latitude": 47.754184
  }
}
```

> Finished chain.


{'request': 'where is city hall?',
 'context': 'Provide your response in JSON',
 'location': 'Seattle, WA',
 'chat_history': [],
 'output': '```json\n{\n  "city_hall_coordinates": {\n    "longitude": -122.279024,\n    "latitude": 47.754184\n  }\n}\n```'}

In [6]:
data_agent.invoke({
    "request": "where is city hall?", 
    "context": "Provide javascript for displaying this location using an ArcGIS Map", #Provide your response in JSON
    "location": "Seattle, WA",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `geocodeLocation` with `{'address': 'City Hall, Seattle, WA'}`


(-122.279024, 47.754184)The coordinates for City Hall in Seattle, WA are:
- Longitude: -122.279024
- Latitude: 47.754184

To display City Hall on an ArcGIS map using JavaScript, you can use the following code:

```javascript
require([
  "esri/Map",
  "esri/views/MapView",
  "esri/geometry/Point",
  "esri/symbols/SimpleMarkerSymbol",
  "esri/Graphic",
  "esri/geometry/SpatialReference"
], function(Map, MapView, Point, SimpleMarkerSymbol, Graphic, SpatialReference) {

  var map = new Map({
    basemap: "streets-navigation-vector"
  });

  var view = new MapView({
    container: "viewDiv",
    map: map,
    center: [47.754184, -122.279024],
    zoom: 15
  });

  var point = new Point({
    longitude: -122.279024,
    latitude: 47.754184
  });

  var markerSymbol = new SimpleMarkerSymbol({
    color: [226, 119, 40],
    outline: {
      color: [255, 255, 255],
      width: 2


{'request': 'where is city hall?',
 'context': 'Provide javascript for displaying this location using an ArcGIS Map',
 'location': 'Seattle, WA',
 'chat_history': [],
 'output': 'The coordinates for City Hall in Seattle, WA are:\n- Longitude: -122.279024\n- Latitude: 47.754184\n\nTo display City Hall on an ArcGIS map using JavaScript, you can use the following code:\n\n```javascript\nrequire([\n  "esri/Map",\n  "esri/views/MapView",\n  "esri/geometry/Point",\n  "esri/symbols/SimpleMarkerSymbol",\n  "esri/Graphic",\n  "esri/geometry/SpatialReference"\n], function(Map, MapView, Point, SimpleMarkerSymbol, Graphic, SpatialReference) {\n\n  var map = new Map({\n    basemap: "streets-navigation-vector"\n  });\n\n  var view = new MapView({\n    container: "viewDiv",\n    map: map,\n    center: [47.754184, -122.279024],\n    zoom: 15\n  });\n\n  var point = new Point({\n    longitude: -122.279024,\n    latitude: 47.754184\n  });\n\n  var markerSymbol = new SimpleMarkerSymbol({\n    color: [226